In [91]:
import polars as pl
import matplotlib.pyplot as plt
import pathlib


In [92]:


ruta = pathlib.Path().parent.resolve()
data = f"{ruta}/data/sapal_salarios_clean.csv"


salarios_sapal =(
    pl.scan_csv(data)
)

df_filtrado = (
    salarios_sapal
    .select([
        pl.col("denominacion_del_cargo").alias("cargo"),
        pl.col("monto_mensual_bruto_de_la_remuneracion_en_tabulador").alias("salario_bruto_mensual")
    ])

)



df_cargos_unicos = (
    df_filtrado
    .select(
        pl.col("cargo")
    )
    .unique()
    .sort("cargo")
    .collect()
    .get_column("cargo")
    .to_list()
)


df_filtrado.collect()


cargo,salario_bruto_mensual
str,f64
"""SECRETARIA""",10416.6
"""PROGRAMADORA ANALISTA ""A""""",16699.0
"""SUP. CATASTRO FISICO INST.""",19532.3
"""AUX. ADMINISTRATIVA REDES""",12696.0
"""EJECUTIVA DEL SISTEMA DE ATENC…",15239.8
…,…
"""CHOFER DE TOLVA""",13921.1
"""CHOFER DE TOLVA""",13921.1
"""VIGILANTE DE POZOS""",13653.9


In [93]:


lf = (
    pl.scan_csv("data/sapal_salarios_clean.csv")
    .select([
        pl.col("denominacion_del_cargo").alias("cargo"),
        pl.col("monto_mensual_bruto_de_la_remuneracion_en_tabulador")
          .alias("salario_bruto_mensual"),
    ])
    .filter(
        pl.col("cargo").is_not_null() & (pl.col("cargo") != "")
    )
    .filter(
        pl.col("salario_bruto_mensual").is_not_null()
    )
)

lf_resumen_cargos = (
    lf
    .group_by("cargo")  # agrupamos en lazy
    .agg([
        pl.len().alias("num_personas_en_cargo"),
        pl.col("salario_bruto_mensual").mean().alias("salario_bruto_promedio"),
        pl.col("salario_bruto_mensual").max().alias("salario_bruto_max"),
        pl.col("salario_bruto_mensual").sum().alias("costo_bruto_total_mensual_cargo"),
    ])
    .sort("salario_bruto_promedio", descending=True)
)

# ahora sí ejecutamos:
df_resumen_cargos = lf_resumen_cargos.collect()
df_resumen_cargos

cargo,num_personas_en_cargo,salario_bruto_promedio,salario_bruto_max,costo_bruto_total_mensual_cargo
str,u32,f64,f64,f64
"""DIRECTOR GENERAL""",1,141667.3,141667.3,141667.3
"""SUBDIRECTOR GENERAL OPERATIVO""",1,101905.4,101905.4,101905.4
"""SUBDIRECTORA DE PLANEACION""",1,90119.6,90119.6,90119.6
"""PROSECRETARIO""",1,86502.0,86502.0,86502.0
"""JEFA DE CONTRALORIA""",1,86502.0,86502.0,86502.0
…,…,…,…,…
"""PEON LABORATORIO""",2,9800.0,9800.0,19600.0
"""PEON ALCANTARILLADO""",24,9800.0,9800.0,235200.0
"""PEON PLANTAS DE TRATAMIENTO""",40,9800.0,9800.0,392000.0


In [94]:
print(f"Total de trabajadores: {df_resumen_cargos['num_personas_en_cargo'].sum()}")

Total de trabajadores: 1605


In [95]:


lista_cargos_importantes = [
    'JEFE DE SISTEMAS COMPUTACIONALES',
    'GERENTE COMERCIAL',
    'DIRECTOR GENERAL',
    'GERENTE DE CALIDAD DEL AGUA Y FISCALIZACION',
    'GERENTE DE SUPERVISION DE OBRA',
    'JEFE DE PROYECTOS',
    'GERENTE DE FINANZAS',
    'GERENTE DE PROYECTOS Y COSTOS',
    'SUBDIRECTORA DE PLANEACION',
    'ADMINISTRADOR DE REDES Y COMUNICACIONES',
    'GERENTE DE AGUA POTABLE Y ALCANTARILLADO',
    'PROGRAMADORA ANALISTA "A"',
    'PROGRAMADOR ANALISTA"B"',
    'PROGRAMADOR ANALISTA',
    'PROGRAMADORA ANALISTA',
    'JEFE DE TECNOLOGIAS DE LA OPERACION',
    'SUBDIRECTOR GENERAL OPERATIVO',
    'JEFE DE PLANEACION HIDRICA',
    'JEFE DE COMUNICACION',
    'JEFE DE COSTOS Y EVALUACION',
    'GERENTE DE TECNOLOGIAS DE LA INFORMACION Y COMUNICACION',
    'GERENTE SERVICIOS ADMINISTRATIVOS'
]


cargo_encontrados = []

for cargo in df_cargos_unicos:
    if cargo  in lista_cargos_importantes:
        cargo_encontrados.append(cargo)
if len(cargo_encontrados) == len(lista_cargos_importantes):
    print(f"Se encontraron todos los cargos importantes: {len(lista_cargos_importantes)}/{len(cargo_encontrados)}")



Se encontraron todos los cargos importantes: 22/22


In [96]:
df_filtrado.columns

/var/folders/j4/21ypxx3x0019wgj7rqnlm2340000gn/T/ipykernel_46234/1013323216.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df_filtrado.columns


['cargo', 'salario_bruto_mensual']